Importing the pickle data and setting up the data to be passed to the modle

In [1]:
#importing data from pickle file
import os
import pickle
import matplotlib.pyplot as plt
os.chdir('..\..\src\models')
from conv_vae import ConvVAE
from conv_vae import ConvolutionalVariationalAutoEncoder
from conv_vsc import ConvVSC
from conv_vsc import ConvolutionalVariationalSparseCoding
os.chdir('..\..\src')
from loading_our_data import BBBC
os.chdir('..\data')

#chose pickle subset
#file_name = "data_100000_images.pkl"
#file_name = "data_10000_images.pkl"
file_name = "data_1000_images.pkl"
data = open (file_name, "rb")
data = pickle.load(data)
print(data.head())

c:\Users\jonat\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


                                              Images   moa
0  [[[tensor(784), tensor(3280), tensor(1072)], [...  DMSO
1  [[[tensor(704), tensor(2656), tensor(992)], [t...  DMSO
2  [[[tensor(2144), tensor(6320), tensor(3072)], ...  DMSO
3  [[[tensor(560), tensor(896), tensor(1520)], [t...  DMSO
4  [[[tensor(832), tensor(4400), tensor(960)], [t...  DMSO


In [2]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
#Creating the data object
dset_train = BBBC(data, "train")
dset_test = BBBC(data, "test")

batch_size = 32
width = 68
height = 68
channels = 3
cuda = True

#check if we need this for somthing form src.utils
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

train_loader = DataLoader(dset_train,batch_size=batch_size, shuffle=True, **kwargs)
test_loader = DataLoader(dset_test,batch_size=batch_size, shuffle=True, **kwargs)
print(f"Length {len(train_loader)}")
print(train_loader)

#images, labels = next(iter(train_loader))
#print(images)

print(type(dset_train))
print(type(dset_train[0]))    #tuple
print(type(dset_train[0][0])) #image
print(type(dset_train[0][1])) #label

print("\n")

print(len(dset_train))        #size of dataset
print(len(dset_train[0]))     #elements in tuple
print(len(dset_train[0][0]))  #length of image

print("\n")

print(dset_train[0][0].type())    #type of image
print(dset_train[0][0][0].type()) #type of one pixel in image

Length 26
<class 'loading_our_data.BBBC'>
in getitem
<class 'tuple'>
in getitem
<class 'torch.Tensor'>
in getitem
<class 'str'>


801
in getitem
2
in getitem
13872


in getitem
torch.FloatTensor
in getitem
torch.FloatTensor


In [3]:
#plot a few examples
f, axarr = plt.subplots(3, 10, figsize=(16, 4))

images, labels = next(iter(train_loader))
print("next itter done")

# Load a batch of images into memory
for i, ax in enumerate(axarr.flat):
    img = images[i]/images[i].max()
    ax.imshow(img.view(68, 68,3))
    ax.axis('off')
        
plt.suptitle('Cells!')
plt.show()

In [3]:
#Setting up model parameters
model_params = {
    'dataset': 'BBBC',
    'width': width,
    'height': height,
    'channels': channels,
    'kernel_szs': '32,68,68',
    'hidden_sz': 256,
    'latent_sz': 200,
    'learning_rate': 1e-3,
    'alpha': 1e-4,
    'device': 'cuda',
    'log_interval': 5000,
    'normalize': False,
    'flatten': False
}

convvsc = ConvolutionalVariationalSparseCoding(**model_params)
